# DeepFactor - Experiments

### Deep Factor (Wang et al.,  2019)

Hybrid method that uses *local* methods that are specific to each time series in the dataset as well as a *global* representation trained on the entire data. The latter is a RNN to learn universal pattern. The current version uses another RNN as local model -> **DeepFactor-RNN (DF-RNN)**

- [`DeepFactorEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deep_factor.html?highlight=deepfactorestimator#gluonts.model.deep_factor.DeepFactorEstimator)
- [Wang et al. (2019)](https://arxiv.org/abs/1905.12417)

In [1]:
# if version does not include DeepState, uncomment the following:
!pip install git+https://github.com/awslabs/gluon-ts.git

import gluonts
print(gluonts.__version__)

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# gluon distribution
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.distribution.gaussian import GaussianOutput

# model imports 
# from gluonts.model.deepstate import DeepStateEstimator
from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

  Cloning https://github.com/awslabs/gluon-ts.git to /tmp/pip-req-build-b3qm33by
  Installing build dependencies ... done
    100% |████████████████████████████████| 81kB 2.0MB/s ta 0:00:011
    100% |████████████████████████████████| 28.4MB 1.9MB/s eta 0:00:011
    100% |████████████████████████████████| 10.4MB 8.1MB/s  eta 0:00:01
    100% |████████████████████████████████| 4.8MB 15.3MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 41.9MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 53.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for gluonts ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-9yfs7ttd/wheels/09/ba/b9/c830628df4d977a9dfe661f6001cdb861fc3f2524bac0ee433
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28

INFO:root:Using CPU


In [2]:
def main(seed=42, data="m4_daily", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepFactorEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         num_layers_global=2,
#         num_layers_local=2
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}

    pprint(output)

# if __name__ == "__main__":
#     simple_main()

In [3]:
%%time
if __name__ == "__main__":
    main(seed=42, data="m4_daily", epochs=100, batches=250)

INFO:root:downloading and processing m4_daily


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 250/250 [00:05<00:00, 44.84it/s, avg_epoch_loss=3.07e+7]
INFO:root:Epoch[0] Elapsed time 5.578 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=30734681.082286
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 250/250 [00:05<00:00, 45.54it/s, avg_epoch_loss=2.91e+6]
INFO:root:Epoch[1] Elapsed time 5.492 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=2912758.646714
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 250/250 [00:05<00:00, 44.73it/s, avg_epoch_loss=8.15e+5]
INFO:root:Epoch[2] Elapsed time 5.592 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=815320.469607
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 250/250 [00:05<00:00, 45.23it/s, avg_epoch_loss=3.71e+5]
INFO:root:Epoch[3] Elapsed time 5.531 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=371136.556643
INFO:root:Epoch[4] Learning rate is 0.001
100%|██████████| 250

GluonTSDataError: Encountered invalid loss value! Try reducing the learning rate or try a different likelihood.